
In this exercise we will determine the towns where a minimum of X% of the stops is located within
the specified radius of the center of the town.

First we determine all the stops that are within the radius of the specified point. (this is identical to ex. 4).

We then determine the towns that the stops are located in.

Lastly we count the stops per town and determine the percentage.

First we initialise PySpark.


In [2]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()


We first read all the stops. The specified file is a preprocessed version of the JSON "stops.txt", in which 
each line contains one stop in the format 

halte_id;halte_name;lat;long;town_name

This makes it easier to parse since it only requires a call to str.split(). The result is an RDD with tuples
(halte_id, halte_name, lat, long, town_name).


In [3]:
stops = sc.textFile("./converted_stops.csv").map(lambda stop: tuple([x.strip() for x in stop.split(";")]))


The user can specify a radius (in meters) by setting the "radius" variable.

The user can select a minimum percentage by setting the "minimum_pct" value (value in [0, 100])


In [4]:
# must be float
# in meters
radius = 300.0

# must be float!
minimum_pct = 20.0

minimum_frac = minimum_pct / 100.0


In order to determine whether we stops are within a certain distance of the town, we need to have the town coordinates.
This CSV file is a preprocessed version of "zipcodes.csv" which filters out useless data. This creates an RDD with tuples (district-name, (lat, long))

We also map the (stop) tuples to (distict-name, stop) tuples. Afterwards we join the two RDDs resulting in tuples
(district-name, (stop, (lat, long)).


In [5]:
town_coords = sc.textFile("./coord_map.csv").map(lambda towncoord: tuple([x.strip() for x in towncoord.split(";")])).map(lambda x: (x[0], (x[1], x[2])))

stops_by_district = stops.keyBy(lambda x: x[4])

stop_with_towncenter = stops_by_district.join(town_coords)


Next, we create a function that determines the distance between the specified point, and a set of coordinates. 

Note: since the earth is a sphere euclidian distances are not 
accurate enough, I have used an online implementation of the haversine method.
http://evoling.net/code/haversine/
https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points/4913653#4913653
There are many sources, I don't know which one is the original.


In [6]:
def haversine(coord1, coord2):
    from math import radians, cos, sin, asin, sqrt
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    lat1 = float(lat1)
    lon1 = float(lon1)
    lat2 = float(lat2)
    lon2 = float(lon2)
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2.0)**2.0 + cos(lat1) * cos(lat2) * sin(dlon/2.0)**2.0
    c = 2.0 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371.0 * c
    m = km * 1000.0
    return m

def get_stop_coord(stop):
  """Retrieve the geo coordinate of a stop."""
  return float(stop[2]), float(stop[3])



We use the function to decide the distance between towns centers and stops.
The (district-name, (stop, (lat, long)) tuples will thus be mapped to (district-name, (stop-within-radius, 1) tuples.

In contrast to previous exercises, we don't apply a filter here since
we need to count both the stops within range and those out of range. Instead, we give each stop
a flag "stop-within-radius". "stop-within-radius" is either 1 for stops that are within range or 0 for stops that are not in range.

The "1" component of the tuple, is so that all the stops can be added together as:

(district-name, (x, y)) + (district-name, (a, b)) = (district-name, (x+a, y+b))

So that the first component determines the amount of stops that are within the radius and the second component determines the total amount of stops.

It would be more orderly to do this mapping step by step in multiple steps, but since that would negatively impact
performance, I did the complete mapping at once.


In [7]:
# (district-name:str, (stop:tuple, in-range:bool))
stops_by_district_with_flag = stop_with_towncenter.map( lambda x : (x[0], (int(haversine(get_stop_coord(x[1][0]), x[1][1]) <= radius),  1 )))


The final step consists of counting the stops that are within the radius and those that are outside of the radius. We use reduce by
key since all items with the same district name need to be grouped together. The reduction itself is done by a pairwise addition.

The result is an RDD with tuples (district-name, (nr-stops-in-range, total-stop-count))


In [8]:

stop_in_range_per_town = stops_by_district_with_flag.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))


Since we want to check which towns conform to a percentage minimum, we need to determine percentages. We will apply a
simple division and apply a check so we don't divide by zero. This results in tuples (town-name, stop-in-range-frac).

These tuples are then filtered so that only towns with enough stops inside the radius are left.


In [9]:
stop_in_range_frac = stop_in_range_per_town.mapValues(lambda x: float(x[0]) / max(float(x[1]), 1))
stop_in_range_frac_filtered = stop_in_range_frac.filter(lambda x: x[1] >= minimum_frac).collect()


Lastly, we print the out for each town.


In [11]:
print("Town - Stop in range pct")
print("------------------------\n")

for town in stop_in_range_frac_filtered:
  print("{}: {}%".format(town[0].encode('utf-8'), town[1] * 100))

Town - Stop in range pct
------------------------

Westkerke: 35.2941176471%
Wulveringem: 30.0%
Doel: 23.0769230769%
Horpmaal: 54.5454545455%
Niel-bij-Sint-Truiden: 30.0%
Sint-Eloois-Vijve: 20.0%
Corswarem: 50.0%
Helkijn: 25.0%
Oostham: 20.0%
Zarren: 22.2222222222%
Westrem: 40.0%
Leisele: 27.7777777778%
Wintershoven: 44.4444444444%
Beverst: 33.3333333333%
Duras: 100.0%
Heldergem: 26.6666666667%
Outrijve: 20.0%
Poederlee: 33.3333333333%
Pollare: 33.3333333333%
Lapscheure: 37.5%
Schore: 50.0%
Rutten: 28.5714285714%
Kessenich: 23.8095238095%
Geraardsbergen: 40.9090909091%
Kobbegem: 22.2222222222%
Sint-Joris-Weert: 28.5714285714%
Bossuit: 66.6666666667%
Glons: 20.0%
Bogaarden: 25.0%
Relegem: 37.5%
Nokere: 60.0%
Zerkegem: 33.3333333333%
Kanne: 45.4545454545%
Groot-Gelmen: 25.0%
Woubrechtegem: 25.0%
Attenhoven: 37.5%
Rukkelingen-Loon: 66.6666666667%
Engelmanshoven: 28.5714285714%
Koolskamp: 26.6666666667%
Bost: 58.3333333333%
Opvelp: 20.0%
Morkhoven: 21.4285714286%
Boorsem: 50.0%
Oorbeek: 33